# Random Forest Regression to Forecast Merchant Monthly Revenue

In [1]:
# Initialise a spark session
import pandas as pd
from collections import Counter
import os
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator


spark = (
    SparkSession.builder.appName("RF Model")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "16g")  # Increase driver memory
    .config("spark.executor.memory", "16g")  # Increase executor memory
    .config("spark.executor.instances", "4")  # Increase the number of executor instances
    .config("spark.driver.maxResultSize", "4g")
    .config("spark.sql.shuffle.partitions", "100") \
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/20 15:08:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.catalog.clearCache()


In [3]:
# Read transaction file
transactions = spark.read.parquet('../data/curated/flagged_fraud')
transactions = transactions.filter(F.col("is_fraud") != True)

In [4]:
transactions.printSchema()

root
 |-- merchant_abn: long (nullable = true)
 |-- year_week: string (nullable = true)
 |-- user_id: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- consumer_id: long (nullable = true)
 |-- fraud_probability_consumer: double (nullable = true)
 |-- name_consumer: string (nullable = true)
 |-- address_consumer: string (nullable = true)
 |-- state_consumer: string (nullable = true)
 |-- postcode_consumer: integer (nullable = true)
 |-- gender_consumer: string (nullable = true)
 |-- name_merchant: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- fraud_probability_merchant: double (nullable = true)
 |-- order_datetime: date (nullable = true)
 |-- order_month_year: string (nullable = true)
 |-- SA4_CODE_2011: string (nullable = true)
 |-- SA4_NAME_2011: string (nullable = true)
 |-- unemployment_rate: string (nullable = true)
 |-- z_score: double (nullable = true)
 |-- consumer_weekly_transaction: long (nullabl

### Feature Engineering

In [5]:
# Aggregating monthly revenue for each merchant
monthly_revenue_df = transactions.groupBy('merchant_abn', 'order_month_year').agg(
    F.sum('dollar_value').alias('monthly_revenue'),
    F.count('order_id').alias('transaction_count'),
    F.avg('fraud_probability_merchant').alias('avg_fraud_probability_merchant'),
    F.first('tags').alias('merchant_tags'),  # Assuming tags and name are constant per merchant
    F.first('name_merchant').alias('merchant_name')
)
    
# Aggregating consumer-level features (most common state and gender for each merchant)

# Most common consumer state per merchant
consumer_state_mode = transactions.groupBy('merchant_abn', 'state_consumer').count() \
    .withColumn('row_num', F.row_number().over(Window.partitionBy('merchant_abn').orderBy(F.desc('count')))) \
    .filter(F.col('row_num') == 1) \
    .select('merchant_abn', 'state_consumer')

# Most common consumer gender per merchant
consumer_gender_mode = transactions.groupBy('merchant_abn', 'gender_consumer').count() \
    .withColumn('row_num', F.row_number().over(Window.partitionBy('merchant_abn').orderBy(F.desc('count')))) \
    .filter(F.col('row_num') == 1) \
    .select('merchant_abn', 'gender_consumer')

# Average Unemployment Rate per Merchant Month-Year
transactions = transactions.withColumn("unemployment_rate_numeric", F.col("unemployment_rate").cast("float"))

unemployment_agg = transactions.groupBy('merchant_abn', 'order_month_year').agg(
    F.avg('unemployment_rate_numeric').alias('avg_unemployment_rate')
)

In [6]:
# Joining Datasets
monthly_revenue_df = monthly_revenue_df.join(consumer_state_mode, on='merchant_abn', how='left') \
                                      .join(consumer_gender_mode, on='merchant_abn', how='left')

# Join with unemployment data on both 'merchant_abn' and 'order_month_year'
monthly_revenue_df = monthly_revenue_df.join(unemployment_agg, on=['merchant_abn', 'order_month_year'], how='left')

# Show the final dataframe
monthly_revenue_df.show(5)

24/09/20 15:08:15 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+------------+----------------+------------------+-----------------+------------------------------+--------------------+--------------------+--------------+---------------+---------------------+
|merchant_abn|order_month_year|   monthly_revenue|transaction_count|avg_fraud_probability_merchant|       merchant_tags|       merchant_name|state_consumer|gender_consumer|avg_unemployment_rate|
+------------+----------------+------------------+-----------------+------------------------------+--------------------+--------------------+--------------+---------------+---------------------+
| 10023283211|          Feb-22|48572.882608193504|              215|            56.069422789172336|((furniture, home...|       Felis Limited|           NSW|           Male|    71.28418544059576|
| 10023283211|          Oct-21| 70276.02432565251|              357|            55.892658715648686|((furniture, home...|       Felis Limited|           NSW|           Male|     79.2271712808048|
| 10142254217|          F

In [7]:
# Checking for Missing Values
nulls = monthly_revenue_df.agg(
    *(F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in monthly_revenue_df.columns)
)

# Show the result
nulls.show()

+------------+----------------+---------------+-----------------+------------------------------+-------------+-------------+--------------+---------------+---------------------+
|merchant_abn|order_month_year|monthly_revenue|transaction_count|avg_fraud_probability_merchant|merchant_tags|merchant_name|state_consumer|gender_consumer|avg_unemployment_rate|
+------------+----------------+---------------+-----------------+------------------------------+-------------+-------------+--------------+---------------+---------------------+
|           0|               0|              0|                0|                             0|            0|            0|             0|              0|                    0|
+------------+----------------+---------------+-----------------+------------------------------+-------------+-------------+--------------+---------------+---------------------+



In [8]:
# Creating lag features to include previous month's revenue
window_spec = Window.partitionBy('merchant_abn').orderBy('order_month_year')

# Lagging features: Previous month's revenue
monthly_revenue_df = monthly_revenue_df.withColumn(
    'previous_month_revenue', F.lag('monthly_revenue', 1).over(window_spec)
)

# Calculate revenue growth (percentage change)
monthly_revenue_df = monthly_revenue_df.withColumn(
    'revenue_growth',
    F.when(F.col('previous_month_revenue') > 0, 
           (F.col('monthly_revenue') - F.col('previous_month_revenue')) / F.col('previous_month_revenue'))
    .otherwise(F.lit(0))  # Fill with 0 if there is no previous revenue
)

# Fill NA values for first month with 0 (no previous data available)
monthly_revenue_df = monthly_revenue_df.fillna({'previous_month_revenue': 0, 'revenue_growth': 0})


monthly_revenue_df = monthly_revenue_df.fillna(0)  # Filling NA values for first month
monthly_revenue_df.show(5)

+------------+----------------+------------------+-----------------+------------------------------+--------------------+-------------+--------------+---------------+---------------------+----------------------+--------------------+
|merchant_abn|order_month_year|   monthly_revenue|transaction_count|avg_fraud_probability_merchant|       merchant_tags|merchant_name|state_consumer|gender_consumer|avg_unemployment_rate|previous_month_revenue|      revenue_growth|
+------------+----------------+------------------+-----------------+------------------------------+--------------------+-------------+--------------+---------------+---------------------+----------------------+--------------------+
| 10023283211|          Apr-21|   9221.4058068711|               47|             56.03849374950703|((furniture, home...|Felis Limited|           NSW|           Male|    74.54042625427246|                   0.0|                 0.0|
| 10023283211|          Aug-21|15807.479921460477|               86|    

### Data Preparation

In [9]:
from pyspark.ml.feature import StandardScaler, StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

# StringIndexing categorical columns (merchant_tags, consumer_state, gender_consumer)
indexers = [
    StringIndexer(inputCol='merchant_tags', outputCol='merchant_tags_indexed', handleInvalid='keep'),
    StringIndexer(inputCol='state_consumer', outputCol='state_consumer_indexed', handleInvalid='keep'),
    StringIndexer(inputCol='gender_consumer', outputCol='gender_consumer_indexed', handleInvalid='keep')
]

# OneHotEncoding indexed columns
encoders = [
    OneHotEncoder(inputCol='merchant_tags_indexed', outputCol='merchant_tags_encoded'),
    OneHotEncoder(inputCol='state_consumer_indexed', outputCol='state_consumer_encoded'),
    OneHotEncoder(inputCol='gender_consumer_indexed', outputCol='gender_consumer_encoded')
]

# VectorAssembler to combine numeric features into a single feature vector
assembler = VectorAssembler(
    inputCols=[
        'monthly_revenue', 'transaction_count', 'avg_fraud_probability_merchant', 'avg_unemployment_rate',
        'merchant_tags_encoded', 'state_consumer_encoded', 'gender_consumer_encoded', 'revenue_growth'
    ], 
    outputCol='features'
)

# Standardizing the numeric features
scaler = StandardScaler(inputCol='features', outputCol='scaled_features')

pipeline = Pipeline(stages=indexers + encoders + [assembler, scaler])

# Fit the pipeline to the dataset
model_pipeline = pipeline.fit(monthly_revenue_df)

final_df = model_pipeline.transform(monthly_revenue_df)

final_df.select('merchant_abn', 'order_month_year', 'scaled_features').show(5)


24/09/20 15:08:56 WARN DAGScheduler: Broadcasting large task binary with size 1443.5 KiB
24/09/20 15:08:57 WARN DAGScheduler: Broadcasting large task binary with size 1433.0 KiB


+------------+----------------+--------------------+
|merchant_abn|order_month_year|     scaled_features|
+------------+----------------+--------------------+
| 10023283211|          Apr-21|(6698,[0,1,2,3,13...|
| 10023283211|          Aug-21|(6698,[0,1,2,3,13...|
| 10023283211|          Dec-21|(6698,[0,1,2,3,13...|
| 10023283211|          Feb-22|(6698,[0,1,2,3,13...|
| 10023283211|          Jan-22|(6698,[0,1,2,3,13...|
+------------+----------------+--------------------+
only showing top 5 rows



### Training Model

In [10]:
train_data, test_data = final_df.randomSplit([0.8, 0.2], seed=42)

In [11]:
rf = RandomForestRegressor(featuresCol='scaled_features', labelCol='monthly_revenue', maxBins=32, maxDepth=10)
rf_model = rf.fit(train_data)

# Make predictions on the test data
predictions = rf_model.transform(test_data)

evaluator = RegressionEvaluator(labelCol='monthly_revenue', predictionCol='prediction', metricName='rmse')

# Root Mean Squared Error (RMSE)
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# R-squared
r2_evaluator = RegressionEvaluator(labelCol='monthly_revenue', predictionCol='prediction', metricName='r2')
r2 = r2_evaluator.evaluate(predictions)
print(f"R-squared: {r2}")

predictions.select('merchant_abn', 'order_month_year', 'monthly_revenue', 'revenue_growth','prediction').show(5)

24/09/20 15:09:09 WARN DAGScheduler: Broadcasting large task binary with size 1869.7 KiB
24/09/20 15:09:09 WARN DAGScheduler: Broadcasting large task binary with size 1869.8 KiB
24/09/20 15:09:09 WARN DAGScheduler: Broadcasting large task binary with size 1940.7 KiB
24/09/20 15:09:11 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
24/09/20 15:09:14 WARN DAGScheduler: Broadcasting large task binary with size 2.3 MiB
24/09/20 15:09:16 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB
24/09/20 15:09:17 WARN DAGScheduler: Broadcasting large task binary with size 3.3 MiB
24/09/20 15:09:19 WARN DAGScheduler: Broadcasting large task binary with size 4.2 MiB
24/09/20 15:09:21 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB
24/09/20 15:09:23 WARN DAGScheduler: Broadcasting large task binary with size 8.1 MiB
24/09/20 15:09:26 WARN DAGScheduler: Broadcasting large task binary with size 11.8 MiB
24/09/20 15:09:29 WARN DAGScheduler: Broadca

Root Mean Squared Error (RMSE): 56834.99204709667


24/09/20 15:09:42 WARN DAGScheduler: Broadcasting large task binary with size 1857.8 KiB


R-squared: 0.7047879286702753


+------------+----------------+------------------+-------------------+------------------+
|merchant_abn|order_month_year|   monthly_revenue|     revenue_growth|        prediction|
+------------+----------------+------------------+-------------------+------------------+
| 10023283211|          Dec-21| 66067.74432316715|  3.179524165232218| 75756.13400048521|
| 10023283211|          Jun-21|11078.327301762967|0.10239164872676415|10698.365621613797|
| 10023283211|          May-21|11953.898144671877| 0.3170441527012353|11584.569867831491|
| 10187291046|          Aug-21|1059.7535269884397|  5.158034850430842| 4346.241413906894|
| 10187291046|          Mar-21| 750.3455540536671| 1.8644858330328316| 2079.050702902205|
+------------+----------------+------------------+-------------------+------------------+
only showing top 5 rows



24/09/20 15:09:44 WARN DAGScheduler: Broadcasting large task binary with size 1854.4 KiB


## Predicting Future Monthly Revenue

In [12]:
from pyspark.sql.types import DateType
from dateutil.relativedelta import relativedelta
from datetime import datetime

# Step 1: Parse the order_month_year column to a proper date format
monthly_revenue_df = monthly_revenue_df.withColumn(
    'order_month_year_date', F.to_date(F.concat(F.lit('01-'), F.col('order_month_year')), 'dd-MMM-yy')
)

# Get the most recent month per merchant
window_spec = Window.partitionBy('merchant_abn').orderBy(F.desc('order_month_year_date'))
latest_merchant_data = monthly_revenue_df.withColumn('row_num', F.row_number().over(window_spec)) \
                                         .filter(F.col('row_num') == 1) \
                                         .drop('row_num')

In [13]:
next_month = 'Aug-24'
future_month_df = spark.createDataFrame([(next_month,)], ['future_order_month_year'])
future_data = latest_merchant_data.crossJoin(future_month_df)

In [14]:
future_data.show(5)

+------------+----------------+------------------+-----------------+------------------------------+--------------------+--------------------+--------------+---------------+---------------------+----------------------+--------------------+---------------------+-----------------------+
|merchant_abn|order_month_year|   monthly_revenue|transaction_count|avg_fraud_probability_merchant|       merchant_tags|       merchant_name|state_consumer|gender_consumer|avg_unemployment_rate|previous_month_revenue|      revenue_growth|order_month_year_date|future_order_month_year|
+------------+----------------+------------------+-----------------+------------------------------+--------------------+--------------------+--------------+---------------+---------------------+----------------------+--------------------+---------------------+-----------------------+
| 10023283211|          Feb-22|48572.882608193504|              215|            56.069422789172336|((furniture, home...|       Felis Limited|    

In [15]:
future_data = model_pipeline.transform(future_data)
future_data = rf_model.transform(future_data)
future_predictions = future_data.select('merchant_abn', 'merchant_name','merchant_tags','future_order_month_year', 'prediction')
future_predictions = future_predictions.withColumnRenamed('prediction', 'projected_revenue')
future_predictions.show(5)

+------------+--------------------+--------------------+-----------------------+------------------+
|merchant_abn|       merchant_name|       merchant_tags|future_order_month_year| projected_revenue|
+------------+--------------------+--------------------+-----------------------+------------------+
| 10023283211|       Felis Limited|((furniture, home...|                 Aug-24|  53665.4148609953|
| 10142254217|Arcu Ac Orci Corp...|([cable, satellit...|                 Aug-24|14747.822730449645|
| 10187291046|Ultricies Digniss...|([wAtch, clock, a...|                 Aug-24|3854.1676478184804|
| 10192359162| Enim Condimentum PC|([music shops - m...|                 Aug-24|10642.865788272718|
| 10206519221|       Fusce Company|[(gift, card, nov...|                 Aug-24| 8170.840440993804|
+------------+--------------------+--------------------+-----------------------+------------------+
only showing top 5 rows



In [16]:
RF_predictions = future_predictions.orderBy(F.col('projected_revenue').desc())

# Show the top 10 merchants by predicted revenue
RF_predictions.show(10)

+------------+--------------------+--------------------+-----------------------+------------------+
|merchant_abn|       merchant_name|       merchant_tags|future_order_month_year| projected_revenue|
+------------+--------------------+--------------------+-----------------------+------------------+
| 32361057556|Orci In Consequat...|([gift, card, nov...|                 Aug-24|1107403.1471705069|
| 43186523025|Lorem Ipsum Sodal...|([florists suppli...|                 Aug-24| 861483.9922429028|
| 89726005175| Est Nunc Consulting|((tent and awning...|                 Aug-24| 800589.5410548949|
| 76767266140|Phasellus At Limited|((furniture, home...|                 Aug-24| 720500.4142418349|
| 86578477987|   Leo In Consulting|[[watch, clock, a...|                 Aug-24| 674400.4402857295|
| 98973094975|   Ornare Fusce Inc.|[(hobby, toy and ...|                 Aug-24| 623676.1192125755|
| 21439773999|Mauris Non Institute|([cable, satellit...|                 Aug-24| 615932.5907567618|


In [17]:
RF_predictions.write.parquet('../data/curated/RF_ranking')

24/09/20 15:13:37 WARN DAGScheduler: Broadcasting large task binary with size 1703.1 KiB


In [18]:
spark.stop()